In [1]:
import pandas as pd
import pprint
import ipywidgets as widgets
from ipywidgets import interact
import matplotlib.pyplot as plt
import math
from pylab import rcParams
rcParams['figure.figsize'] = 25, 13

In [2]:
from pymongo import MongoClient

from IPython.core.display import display, HTML
[display(HTML("<style>.container { width:100% !important; }</style>"))];

In [3]:
with MongoClient('mongodb://localhost:27017/') as client:
    database = client["nyc_crashes"]
    crashes = database.crashes
    crashes_by_year = database.crashes_by_year

In [67]:
def yearly(Year):
    data = crashes_by_year.find_one({"_id": Year})['by_month']
    
    x_values = data.keys()
    y_values_injured = []
    y_values_killed = []
    for month in data:
        injured = 0
        killed = 0
        injured = data[month]['pedestrians']['injured'] + data[month]['cyclists']['injured'] + data[month]['motorists']['injured']
        killed = data[month]['pedestrians']['killed'] + data[month]['cyclists']['killed'] + data[month]['motorists']['killed']
        y_values_injured += [injured]
        y_values_killed += [killed]
    
    show_graph(x_values, y_values_injured, y_values_killed)
    return

In [54]:
def show_graph(x_values, y_values_injured, y_values_killed):
    plt.figure(figsize=(20,10))

    plt.bar(x_values, y_values_killed, color=(0.62,0.07,0.04))
    plt.bar(x_values, y_values_injured, bottom=y_values_killed, color=(0.80,0.52,0.00))
    plt.show()

In [76]:
interact(yearly, Year=[data['_id'] for data in crashes_by_year.find().sort('_id', -1)]);

interactive(children=(Dropdown(description='Year', options=('2020', '2019', '2018', '2016', '2013'), value='20…

In [38]:
def monthly(Month):
    
    return Month

In [44]:
def out2(Year):
    #return interact(out3, Month=list(range(1, 13)));
    #return interact(out3, Month=(1,12));
    #return interact(monthly, Month=widgets.IntSlider(min=1,max=12,step=1,value=1))
    dates = [datetime.date(2015, i, 1) for i in range(1, 13)]
    options = [(i.strftime('%b'), i) for i in dates]
    widgets.SelectionRangeSlider(
        options=options,
        index=(0, 11),
        description='2015',
        disabled=False
    )
    return

In [45]:
import datetime
interact(out2, Year=[data['_id'] for data in crashes_by_year.find()]);

interactive(children=(Dropdown(description='Year', options=('2019', '2020', '2013', '2016', '2018'), value='20…

In [30]:
from datetime import datetime

start_date = datetime(2018, 4, 24)
end_date = datetime(2018, 5, 24)

dates = pd.date_range(start_date, end_date, freq='B')

options = [(date.strftime('%b %Y '), date) for date in dates]
index = (0, len(options)-1)

selection_range_slider = widgets.SelectionRangeSlider(
    options=options,
    index=index,
    description='Dates',
    orientation='horizontal',
    layout={'width': '500px'}
)

selection_range_slider

SelectionRangeSlider(description='Dates', index=(0, 22), layout=Layout(width='500px'), options=(('Apr 2018 ', …

In [32]:
import datetime
dates = [datetime.date(2015, i, 1) for i in range(1, 13)]
options = [(i.strftime('%b'), i) for i in dates]
widgets.SelectionRangeSlider(
    options=options,
    index=(0, 11),
    description='2015',
    disabled=False
)

SelectionRangeSlider(description='2015', index=(0, 11), options=(('Jan', datetime.date(2015, 1, 1)), ('Feb', d…